## Benchmarks de AIONER

### Clonación del Repositorio

In [ ]:
!bash ./AIONER/setup.sh

In [2]:
import os
from datasets_utils import datasets_map
from ner_benchmark_data_wrangler import NERBenchmarkDataWrangler
from utils import create_directories, save_to_file
import subprocess

In [ ]:
initial_dirs = ['./custom_datasets', './custom_vocab', './metrics']

create_directories(initial_dirs)

for dataset in datasets_map.keys():
    wrangler = NERBenchmarkDataWrangler(dataset, datasets_map)
    
    train_df = wrangler.get_format_data('train')
    valid_df = wrangler.get_format_data('valid')
    test_df = wrangler.get_format_data('test')
    
    train_pubtator = wrangler.to_pubtator('train')
    test_pubtator = wrangler.to_pubtator('test')
    valid_pubtator = wrangler.to_pubtator('valid')
    
    labels = list(set(train_df['labels'].unique().tolist() +
                      valid_df['labels'].unique().tolist() +
                      test_df['labels'].unique().tolist()))
    
    dataset_dirs = [f'./custom_datasets/{dataset}', f'./custom_vocab/{dataset}']

    create_directories(dataset_dirs)
    
    save_to_file(f'./custom_datasets/{dataset}/train.txt', train_pubtator)
    save_to_file(f'./custom_vocab/{dataset}.txt', labels)
    save_to_file(f'./custom_datasets/{dataset}/test.txt', test_pubtator)
    save_to_file(f'./custom_datasets/{dataset}/valid.txt', valid_pubtator)

In [ ]:
for dataset in datasets_map.keys():
    subprocess.run(
        f'python AIONER/src/AIONER_Run.py -i ./custom_datasets/{dataset}/ -m AIONER/pretrained_models/AIONER/Bioformer-softmax-AIONER.h5 -v ./custom_vocab/{dataset}.txt -e ALL -o output/{dataset}/',
        shell=True
    )

    subprocess.run(
        f'python AIONER/src/AIONER_Eval.py --gold ./custom_datasets/{dataset}/test.txt --pred output/{dataset}/test.txt',
        shell=True
    )